In [1]:
import os
import openai
import sys
from langchain.chat_models import ChatOpenAI
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.environ['OPENAI_API_KEY']


In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'

In [3]:
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

/var/folders/7t/ys9t4mvn1fx74wckbywxd2sw0000gp/T/ipykernel_15979/2897838732.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()
/var/folders/7t/ys9t4mvn1fx74wckbywxd2sw0000gp/T/ipykernel_15979/2897838732.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [4]:
print(vectordb._collection.count())

298


In [5]:
question = "what is logistic regression?"

In [6]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=2, fetch_k=3)

In [7]:
vectordb.similarity_search(question,k=3)

[Document(metadata={'author': 'Sepp Hochreiter', 'creationdate': '2024-01-19T16:06:01+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2024-01-19T16:06:01+00:00', 'page': 140, 'page_label': '129', 'producer': 'pdfTeX-1.40.25', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': 'Script_Machine Learning Basic Techniques.pdf', 'subject': '', 'title': 'Basic Methods of Data Analysis', 'total_pages': 175, 'trapped': '/False'}, page_content='7.4. Binary Models 129\nIf x is in the k-th class then y = (0 , . . . ,0, 1, 0, . . . ,0), where the “1” is at position k in the\nvector y.\nThe likelihood of iid data is\nL({z}; w) = p({z}; w) =\nNY\nn=1\nKY\nk=1\np(yn = ek | xn; w)[yn]k p(xn) = (7.34)\nNY\nn=1\np(xn)\nKY\nk=1\np(yn = ek | xn; w)[yn]k\nbecause\nKY\nk=1\np(yn = ek | xn; w)[yn]k = p(yn = er | xn; w) for yn = er . (7.35)\nThe log-likelihood is\nln L({z}; w) =\nKX\nk=1\nNX\nn=1\n[yn]k ln p(yn = ek | xn;

In [8]:
docs_mmr[0].page_content[:100]

'7.4. Binary Models 129\nIf x is in the k-th class then y = (0 , . . . ,0, 1, 0, . . . ,0), where the '

In [9]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [10]:
document_content_description = "Machine Learning Basic Techniques"
llm = ChatOpenAI(model='gpt-4o', temperature=0)
# retriever = SelfQueryRetriever.from_llm(
#     llm,
#     vectordb,
#     document_content_description,
#     verbose=True
# )

/var/folders/7t/ys9t4mvn1fx74wckbywxd2sw0000gp/T/ipykernel_15979/16750302.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model='gpt-4o', temperature=0)


In [11]:
#docs = retriever.get_relevant_documents(question)

In [12]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [13]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [14]:
llm = ChatOpenAI(temperature=0, model="o3-mini")
compressor = LLMChainExtractor.from_llm(llm)

In [15]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [16]:
compressed_docs = compression_retriever.get_relevant_documents(question)
print (compressed_docs)
pretty_print_docs(compressed_docs)

/var/folders/7t/ys9t4mvn1fx74wckbywxd2sw0000gp/T/ipykernel_15979/1955647316.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  compressed_docs = compression_retriever.get_relevant_documents(question)


[Document(metadata={'author': 'Sepp Hochreiter', 'creationdate': '2024-01-19T16:06:01+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2024-01-19T16:06:01+00:00', 'page': 140, 'page_label': '129', 'producer': 'pdfTeX-1.40.25', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': 'Script_Machine Learning Basic Techniques.pdf', 'subject': '', 'title': 'Basic Methods of Data Analysis', 'total_pages': 175, 'trapped': '/False'}, page_content='A function g mapping x onto R can be transformed into a probability by the sigmoidal function 1 1 + e− g(x;w) (7.38) which is depicted in Fig. 7.2. Note that 1 − 1 1 + e− g(x;w) = e− g(x;w) 1 + e− g(x;w) . (7.39) We set p(y = 1 | x; w) = 1 1 + e− g(x;w) (7.40)')]
Document 1:

A function g mapping x onto R can be transformed into a probability by the sigmoidal function 1 1 + e− g(x;w) (7.38) which is depicted in Fig. 7.2. Note that 1 − 1 1 + e− g(x;w) = e− g(x;w) 1 + 